In [1]:
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import scipy as sp
import statsmodels.formula.api as sm
from sklearn.inspection import permutation_importance
from itertools import product

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#no dado original, havia uma coluna sem nome (com os indexes). seria difícil remover assim. Nomeei-a de "A", para priorizar 
#o index do pandas.

# Função para normalizar todas as colunas de um dataframe pelo método de valor mínimo 0 e máximo 1
def normalizar(df):
    resultado = df.copy()
    for coluna in df.columns:
        if df[coluna].max() == 0:
            continue
        else:
            valor_max = df[coluna].max()
            valor_min = df[coluna].min()
            resultado[coluna] = (df[coluna] - valor_min) / (valor_max - valor_min)
    return resultado


In [3]:
#criando o DataFrame "df_chem"
df = pd.read_csv("Minerals_Database3.csv")  # Importação do dataset para ser utilizado como dataframe
# Remoção de colunas não necessárias para o dataframe

df = df.drop(['A'], axis=1)
df = df.drop(['Name'], axis=1)

dados_cat = df.reindex(df.columns[[0,2,4]], axis = 1) # Remoção de dados categóricos
dados_categoricos = dados_cat.astype("category") # Tratamento de dados para o tipo categórico
newlist = [x for x in range(7,135)] # Range para pegarmos dados que estão em porcentagem
porcent_df = df.reindex(df.columns[newlist], axis = 1) # Coleta dos dados de elementos em porcentagem
df_chem = df.reindex(df.columns[[1,3,5,135,136,137]], axis=1) # Reindexação de um dataframe contendo apenas propriedades fisico-químicas
df_chem
df_chem, dados_categoricos

df_remove=df_chem.loc[(df_chem["Refractive Index"] == 0.000)] #remove as linhas (objetos), cujo valor do índice de refração é 0

df_chem = df_chem.drop(df_remove.index) #considera que o dataframe df_chem é o dataframe com as linhas removidas
print(df_chem)

     Mohs Hardness  Specific Gravity  Refractive Index  Molar Mass  \
0             4.50             3.240             1.580  817.339002   
1             2.75             3.446             1.592  435.069330   
2             2.00             4.420             2.085  921.092220   
4             5.50             1.050             1.634  861.185368   
5             3.50             3.295             1.457  225.618151   
..             ...               ...               ...         ...   
803           7.50             4.650             1.928  526.041800   
805           6.50             3.230             1.702  379.378178   
806           3.50             2.180             1.590  492.887716   
807           7.00             3.411             1.596  263.580584   
808           0.00             3.146             1.633  576.801907   

     Molar Volume  Calculated Density  
0        0.123390               5.498  
1        0.056083               6.439  
2        0.122631               6.234  

In [4]:
# Computação e análise descritiva e estatística dos dados

# Valores de média, desvio padrão, mínimos e máximos, Q1, mediana e Q3
print(df_chem.describe())

# Moda do dataframe
df_chem.mode

       Mohs Hardness  Specific Gravity  Refractive Index   Molar Mass  \
count     588.000000        588.000000        588.000000   588.000000   
mean        3.879779          3.067937          1.712794   474.243503   
std         1.964125          1.552768          0.241588   470.660380   
min         0.000000          0.000000          0.989000    25.011580   
25%         2.500000          2.394750          1.570000   190.132605   
50%         3.500000          3.050000          1.646000   330.319673   
75%         5.500000          3.750500          1.773000   573.779844   
max         9.500000          9.593000          3.383000  4079.341200   

       Molar Volume  Calculated Density  
count    588.000000          588.000000  
mean       0.099284            4.793639  
std        0.102466            2.706052  
min        0.011040            0.635000  
25%        0.033789            2.778750  
50%        0.067282            4.145000  
75%        0.123304            6.240250  
max   

<bound method DataFrame.mode of      Mohs Hardness  Specific Gravity  Refractive Index  Molar Mass  \
0             4.50             3.240             1.580  817.339002   
1             2.75             3.446             1.592  435.069330   
2             2.00             4.420             2.085  921.092220   
4             5.50             1.050             1.634  861.185368   
5             3.50             3.295             1.457  225.618151   
..             ...               ...               ...         ...   
803           7.50             4.650             1.928  526.041800   
805           6.50             3.230             1.702  379.378178   
806           3.50             2.180             1.590  492.887716   
807           7.00             3.411             1.596  263.580584   
808           0.00             3.146             1.633  576.801907   

     Molar Volume  Calculated Density  
0        0.123390               5.498  
1        0.056083               6.439  
2      

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

tamanho = 0.25 # Fração de dados escolhida para treino e teste
seed = 2000

i = df_chem.index
i_treino, i_teste = train_test_split(i, test_size=tamanho, random_state = seed)

df_treino = df_chem.loc[i_treino]
df_teste = df_chem.loc[i_teste]

#print(df_treino)
#print()
#print(df_teste)

In [6]:
# Modelo preditivo
# Definições de atributos e targets
nome = 'df_chem'
atributos = ["Specific Gravity", "Mohs Hardness", 'Molar Mass', 'Molar Volume', 'Calculated Density']
target = ["Refractive Index"]

X_treino = df_treino.reindex(atributos, axis=1).values
y_treino = df_treino.reindex(target, axis=1).values
X_teste = df_teste.reindex(atributos, axis=1).values
y_teste = df_teste.reindex(target, axis=1).values

### Baseline

In [7]:
modelo_baseline = DummyRegressor()# Criação do Modelo fictício

modelo_baseline.fit(X_treino, y_treino) # Treino do Modelo fictício

previsao = modelo_baseline.predict(X_teste) # Previsão do modelo a partir do treino
print(previsao)

[1.71836735 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735
 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735
 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735
 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735
 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735
 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735
 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735
 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735
 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735
 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735
 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735
 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735
 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735
 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735
 1.71836735 1.71836735 1.71836735 1.71836735 1.71836735 1.7183

In [8]:
y_verdadeiro = y_teste
y_previsao = modelo_baseline.predict(X_teste)
RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False) # Método da raiz quadrada do erro quadrático médio
print(f'A RMSE do modelo de baseline foi uma Indice Refrativo de {RMSE: .4f}')
#print(y_verdadeiro)

A RMSE do modelo de baseline foi uma Indice Refrativo de  0.2030


In [9]:
tamanho = 0.25 # Fração de dados escolhida para treino e teste
seed = 567

i = df_chem.index
i_treino, i_teste = train_test_split(i, test_size=tamanho, random_state = seed) #Split entre dados treino e teste

df_treino = df_chem.loc[i_treino]
df_teste = df_chem.loc[i_teste]

print(df_treino)
print()
print(df_teste)

     Mohs Hardness  Specific Gravity  Refractive Index   Molar Mass  \
615           0.00             5.700             1.737   298.038100   
194           5.50             4.650             2.120   123.842500   
306           4.50             2.285             1.516   257.230046   
82            0.00             4.380             1.793   426.292200   
339           3.00             6.420             2.358  1560.452603   
..             ...               ...               ...          ...   
64            2.50             1.715             1.513   155.071500   
29            6.25             2.625             1.535   411.331124   
762           5.50             0.000             1.495   340.284075   
367           1.00             3.197             1.641   490.595370   
250           5.00             4.535             2.069   656.503600   

     Molar Volume  Calculated Density  
615      0.033606               7.361  
194      0.011210               9.169  
306      0.033718          

In [12]:
# Modelo preditivo
# Definições de atributos e targets
nome = 'df_chem'
atributos = ["Specific Gravity", "Mohs Hardness", 'Molar Mass', 'Molar Volume', 'Calculated Density']
target = ["Refractive Index"]

X_treino = df_treino.reindex(atributos, axis=1).values
y_treino = df_treino.reindex(target, axis=1).values
X_teste = df_teste.reindex(atributos, axis=1).values
y_teste = df_teste.reindex(target, axis=1).values

### K-NN

In [13]:
from sklearn.neighbors import KNeighborsRegressor

# criação do modelo K-NN
modelo_knn = KNeighborsRegressor()

# treinamento do modelo
modelo_knn.fit(X_treino, y_treino)

# realiza uma previsão usando o modelo treinado
previsao = modelo_knn.predict(X_teste)
print(previsao)

[[1.8666]
 [1.5868]
 [1.5968]
 [1.8562]
 [1.7724]
 [1.574 ]
 [1.6042]
 [1.6064]
 [1.8562]
 [1.7314]
 [1.554 ]
 [1.9476]
 [1.6026]
 [1.643 ]
 [1.7646]
 [1.792 ]
 [1.6702]
 [1.4204]
 [1.6222]
 [1.6072]
 [1.813 ]
 [1.7136]
 [1.807 ]
 [1.6184]
 [1.589 ]
 [1.8672]
 [1.8448]
 [1.5962]
 [1.6896]
 [1.6272]
 [1.9178]
 [1.5988]
 [1.6378]
 [1.771 ]
 [1.6404]
 [1.8344]
 [1.5968]
 [1.6272]
 [1.4204]
 [1.759 ]
 [1.6938]
 [1.5968]
 [1.6378]
 [1.753 ]
 [1.721 ]
 [1.9646]
 [1.6222]
 [1.709 ]
 [1.6196]
 [1.8744]
 [1.893 ]
 [1.595 ]
 [1.7308]
 [1.6036]
 [1.6368]
 [1.7366]
 [1.6572]
 [1.7726]
 [1.7952]
 [1.609 ]
 [1.5944]
 [1.7394]
 [1.6784]
 [1.5968]
 [1.6306]
 [1.7524]
 [1.7736]
 [1.596 ]
 [1.7232]
 [1.776 ]
 [1.7136]
 [1.6152]
 [1.6248]
 [1.617 ]
 [1.6288]
 [1.6024]
 [1.6528]
 [1.6378]
 [1.6842]
 [1.6688]
 [1.6484]
 [1.7598]
 [1.549 ]
 [1.6404]
 [1.8698]
 [1.768 ]
 [1.789 ]
 [1.74  ]
 [1.6916]
 [1.7388]
 [1.7642]
 [1.8498]
 [1.723 ]
 [1.617 ]
 [1.5636]
 [1.7108]
 [1.8994]
 [1.7454]
 [1.6354]
 [1.8324]


In [14]:
y_verdadeiro = y_teste
y_previsao = modelo_knn.predict(X_teste)

RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False)

print(f"O RMSE do modelo k-NN foi um Indice Refrativo de {RMSE}.")

O RMSE do modelo k-NN foi um Indice Refrativo de 0.25713131085188395.


### Árvore de Decisões

In [15]:
from sklearn.tree import DecisionTreeRegressor

# cria o modelo
modelo_dt = DecisionTreeRegressor(random_state=1024)

# treina o modelo
modelo_dt.fit(X_treino, y_treino)

# realiza uma previsão usando o modelo treinado
previsao = modelo_dt.predict(X_teste)
#print(previsao)

In [16]:
from sklearn.metrics import mean_squared_error

y_verdadeiro = y_teste
y_previsao = modelo_dt.predict(X_teste)

RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False)

print(f"O RMSE do modelo árvore de decisão foi de {RMSE}.")

O RMSE do modelo árvore de decisão foi de 0.25762351901761693.


### Random Forest

In [17]:
#Treinamento do modelo
from sklearn.ensemble import RandomForestRegressor

# alterando o formato do target
print(y_treino.shape, y_teste.shape)
y_treino = y_treino.ravel()
y_teste = y_teste.ravel()
print(y_treino.shape, y_teste.shape)

# cria o modelo
modelo_rf = RandomForestRegressor(random_state=1024)

# treina o modelo
modelo_rf.fit(X_treino, y_treino)

# realiza uma previsão usando o modelo treinado
previsao = modelo_rf.predict(X_teste)
print()
print(previsao)

(441, 1) (147, 1)
(441,) (147,)

[1.71462 1.5677  1.63808 1.59898 1.5532  1.69809 1.6582  1.69229 1.70891
 1.66317 1.50641 1.7545  1.62664 2.0001  1.53531 1.74324 1.77298 1.42581
 1.61285 1.52606 1.62843 2.09241 1.63166 1.63625 1.55433 2.14116 1.84656
 1.5969  1.72575 1.76197 1.6316  1.63442 1.75772 1.64072 1.58994 1.53642
 1.56924 1.71102 2.40109 1.69281 1.9513  1.69493 1.57237 1.64316 1.75887
 1.63692 1.64068 1.54083 1.67168 2.10355 1.5684  1.63454 2.09757 1.53205
 1.63949 1.62289 1.70955 1.54317 1.72294 1.46641 1.53247 1.72221 1.66515
 1.67223 1.53466 1.7105  2.18203 1.4954  1.77879 1.67679 1.70481 1.74677
 2.12201 1.61541 1.88213 1.50512 1.6259  1.5051  1.63998 1.55489 1.70768
 1.92539 1.79923 1.70376 2.0278  1.71737 2.16094 1.69216 1.56685 1.52425
 1.49403 1.57961 1.67683 1.63089 2.28097 1.60658 1.5566  1.69387 1.69675
 2.79491 1.53922 1.69973 1.69923 1.83632 1.55604 1.55831 1.53225 1.47652
 1.58315 1.56188 1.52015 1.58985 1.71298 1.77794 1.79485 1.62914 1.6324
 1.53511 1.68809 1.

In [18]:
#Performance do modelo de floresta aleatória
from sklearn.metrics import mean_squared_error

y_verdadeiro = y_teste
y_previsao = modelo_rf.predict(X_teste)

RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False)

print(f"O RMSE do modelo de floresta aleatória foi de {RMSE}")

O RMSE do modelo de floresta aleatória foi de 0.22686987805503722


### Regressão Linear

In [19]:
# Modelo linear para compararmos com nossa Baseline

linear = LinearRegression() # Criação do Modelo Linear

linear.fit(X_treino, y_treino) # Treinamento do Modelo Linear

previsao_linear = linear.predict(X_teste) # Previsão do modelo linear a partir do treino

#print(previsao_linear)

In [20]:
# Comparação se foi melhor ou não

y_verdadeiro = y_teste
y_previsao = linear.predict(X_teste)

RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False)
print(f'A RMSE do modelo linear foi uma Indice Refrativo de {RMSE}')

A RMSE do modelo linear foi uma Indice Refrativo de 0.2354673749765964


Desse modo, pode-se concluir que o melhor modelo possível para a previsão do Índice Refrativo é a Baseline, uma vez que comparado a todos os outros modelos apresenta o menor RMSE. Isso pode indicar que há um problema com o modelo, uma vez que "baseline" não é um modelo particularmente sofisticado.